In [5]:
import os
from typing import TypedDict, Annotated
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
load_dotenv()
from langchain_community.tools.tavily_search import TavilySearchResults
google_api_key = os.getenv("GOOGLE_API_KEY")
tool=TavilySearchResults(max_results=2)

tools = [tool]

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=2, name="tavily_search")
tools = [search_tool]


class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

def call_model(state: AgentState):
    """The 'agent' node: calls the model to decide the next action."""
    print("---AGENT: CALLING MODEL---")
    messages = state['messages']

    response = model.invoke(messages, tools=tools)
    # The response, which may contain tool calls, is returned to be added to the state
    return {"messages": [response]}

tool_node = ToolNode(tools)

# --- 4. DEFINE ROUTER (CONDITIONAL EDGE) ---

def should_continue(state: AgentState) -> str:
    """
    Router logic: Inspects the last message to see if it contains tool calls.
    This function is made more robust by using `getattr` to safely access
    the .tool_calls attribute.
    """
    print("---ROUTER: CHECKING FOR NEXT STEP---")
    last_message = state['messages'][-1]
    
    # Safely check for tool_calls attribute
    if getattr(last_message, "tool_calls", None):
        print(">>> Decision: Tool call detected, routing to action.")
        return "action"
    else:
        print(">>> Decision: No tool call, routing to end.")
        return "end"

# --- 5. CONSTRUCT THE GRAPH ---
# The graph structure is identical to before

graph_builder = StateGraph(AgentState)

# Add the two nodes we defined: the agent and the tool executor
graph_builder.add_node("agent", call_model)
graph_builder.add_node("action", tool_node)

# The agent node is our entry point
graph_builder.set_entry_point("agent")

# Add the conditional edge for routing
graph_builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "action": "action",
        "end": END,
    },
)

# The action node loops back to the agent node
graph_builder.add_edge("action", "agent")

# Compile the graph into a runnable object
graph = graph_builder.compile()



In [21]:
inputs = {"messages": [HumanMessage(content="What is the current price of the NVIDIA stock?")]}

print("---STARTING GRAPH EXECUTION---")
for event in graph.stream(inputs, stream_mode="values"):
    messages = event["messages"]
    last_message = messages[-1]
    
    if "tool_calls" in last_message.additional_kwargs:
        print(f"\nTOOL CALL: {last_message.tool_calls}")
    else:
        print(f"\nFINAL ANSWER:\n{last_message.content}")

print("\n---GRAPH EXECUTION FINISHED---")

---STARTING GRAPH EXECUTION---

FINAL ANSWER:
What is the current price of the NVIDIA stock?
---AGENT: CALLING MODEL---
---ROUTER: CHECKING FOR NEXT STEP---
>>> Decision: Tool call detected, routing to action.

FINAL ANSWER:


FINAL ANSWER:
[{"title": "NVIDIA Stock Chart — NASDAQ:NVDA Stock Price - TradingView", "url": "https://www.tradingview.com/symbols/NASDAQ-NVDA/", "content": "See all sparks\n\nFrequently Asked Questions\n--------------------------\n\nWhat is NVIDIA stock price today?\n\nThe current price of NVDA is 163.05 USD — it has increased by 1.91% in the past 24 hours. Watch NVIDIA stock price performance more closely on the chart.\n\nWhat is NVIDIA stock ticker?\n\nDepending on the exchange, the stock ticker may vary. For instance, on NASDAQ exchange NVIDIA stocks are traded under the ticker NVDA.\n\nIs NVIDIA stock price growing?", "score": 0.9301101}, {"title": "NVIDIA - 26 Year Stock Price History | NVDA - Macrotrends", "url": "https://www.macrotrends.net/stocks/charts/